In [1]:
import os, sys
import numpy as np
import pandas as pd
import copy

things_root = '/user_data/mmhender/stimuli/things/'
project_root = '/user_data/mmhender/featsynth/'
stimuli_folder = 'images_v3'
ecoset_info_path = '/user_data/mmhender/stimuli/ecoset_info/'

sys.path.insert(0, '/user_data/mmhender/featsynth/code/')
from utils import dropbox_utils, expt_utils
token_path = os.path.join(project_root, 'tokens/dbx_token.txt')
dbx = dropbox_utils.init_dropbox(token_path)

expt_name = 'expt4'

# mini-block design

In [2]:
rndseed=132434

In [3]:
# using here the same set of categories that we chose from ecoset.
# 64 basic categories in 8 superordinate groups.
fn = os.path.join(ecoset_info_path, 'categ_use_ecoset.npy')
info = np.load(fn, allow_pickle=True).item()
basic_names = list(info['binfo'].keys())
super_names = list(info['sinfo'].keys())

# load info about things images/names 
fn2load = os.path.join(things_root, 'things_file_info.npy')
tfiles = np.load(fn2load, allow_pickle=True).item()

categ_sets = [super_names]
concept_sets = [[info['sinfo'][sname]['basic_names'] for sname in super_names]]

n_ex_use = 10;
n_counterbalance_conds = 1


# figure out trial conditions/counts
n_categ_use = len(categ_sets[0])
n_concepts_use = len(concept_sets[0][0])
# n_ex_use = len(image_name_sets[0][0][0])

# image types are original or "synth" from different DNN layers
n_layers=4
n_image_types = n_layers+1 

cue_levels = ['basic', 'super']
n_cue_levels = len(cue_levels); # basic or superordinate

n_trials_per_concept = n_image_types * n_cue_levels
assert(n_trials_per_concept==n_ex_use)

n_trials_total = n_categ_use * n_concepts_use * n_trials_per_concept

n_runs = 16;
assert(np.mod(n_trials_total, n_runs)==0)
n_trials_per_run = int(n_trials_total/n_runs);

n_trials_mini = 10

assert(np.mod(n_trials_per_run, n_trials_mini)==0)
n_mini_per_run = int(n_trials_per_run/n_trials_mini)

n_mini_total = n_mini_per_run * n_runs

n_trials_per_categ = n_trials_per_run # this just happens to work out this way
n_mini_per_categ = int(n_trials_per_categ / n_trials_mini)

np.random.seed(rndseed)


In [4]:
cb = 0

print('making trial info for counterbalance cond %d of %d'%(cb, n_counterbalance_conds))

categ_use = categ_sets[cb]
categ_use = [c.replace(' ', '_') for c in categ_use]
print(categ_use)
concepts_use = concept_sets[cb]
# image_names_use = image_name_sets[cb]

# make 100 different randomized orders for this same image set
# n_random_orders=2;
n_random_orders=100;

trial_info_list = []

dbxpath_dict = dict()


making trial info for counterbalance cond 0 of 1
['insect', 'vegetable', 'fruit', 'dessert', 'tool', 'musical_instrument', 'furniture', 'vehicle']


In [5]:
rand = 0

trial_info = pd.DataFrame({'trial_num_overall': np.zeros((n_trials_total,)), 
                           'trial_in_run': np.zeros((n_trials_total,)), 
                           'trial_in_miniblock': np.zeros((n_trials_total,)), 
                            'run_number': np.zeros((n_trials_total,)), 
                            'miniblock_number_overall': np.zeros((n_trials_total,)), 
                            'miniblock_number_in_run': np.zeros((n_trials_total,)), 
                            'image_set_num': np.zeros((n_trials_total,)), 
                            'random_order_number': np.zeros((n_trials_total,)), 
                           'categ_ind': np.zeros((n_trials_total,)),
                          'concept_ind': np.zeros((n_trials_total,)),
                          'super_name': np.zeros((n_trials_total,)),
                          'basic_name': np.zeros((n_trials_total,)),
                          'ex_num': np.zeros((n_trials_total,)),
                          'image_type_num': np.zeros((n_trials_total,)),
                          'image_type': np.zeros((n_trials_total,)),
                          'image_name': np.zeros((n_trials_total,)),
                          'dropbox_url': np.zeros((n_trials_total,)),
                          'cue_level_num': np.zeros((n_trials_total,)),
                          'cue_level': np.zeros((n_trials_total,)),
                          'cue_name': np.zeros((n_trials_total,)),
                          'distractor_name': np.zeros((n_trials_total,)),
                          'left_name': np.zeros((n_trials_total,)),
                          'right_name': np.zeros((n_trials_total,)),
                          'correct_resp': np.zeros((n_trials_total,)), 
                          })

image_type_names = ['orig', 'pool1','pool2','pool3','pool4']

tt=-1
for ca in range(n_categ_use):

    for co in range(n_concepts_use):

        ex = -1

        for typ in range(n_image_types):
            for cue in range(n_cue_levels):

                tt+=1
                if np.mod(tt,100)==0:
                    print('proc trial %d of %d'%(tt, n_trials_total))

                trial_info['trial_num_overall'].iloc[tt] = tt
                trial_info['categ_ind'].iloc[tt] = ca
                trial_info['concept_ind'].iloc[tt] = co
                trial_info['super_name'].iloc[tt] = categ_use[ca]
                trial_info['basic_name'].iloc[tt] = concepts_use[ca][co].replace(' ', '_')

                trial_info['image_type_num'].iloc[tt] = typ
                trial_info['image_type'].iloc[tt] = image_type_names[typ]
                trial_info['cue_level_num'].iloc[tt] = cue
                trial_info['cue_level'].iloc[tt] = cue_levels[cue]

                trial_info['image_set_num'].iloc[tt] = cb
                trial_info['random_order_number'].iloc[tt] = rand

                ex += 1
                trial_info['ex_num'].iloc[tt] = ex

                bname = concepts_use[ca][co]
                name_raw = tfiles[bname][ex].split('.jpg')[0]
                
                # name_raw = image_names_use[ca][co][ex].split('.jpg')[0]

                if image_type_names[typ]=='orig':

                    trial_info['image_name'].iloc[tt] = os.path.join(name_raw, 'orig.png')
                    dbxpath = '/'+os.path.join(stimuli_folder, name_raw, 'orig.png')

                else:

                    trial_info['image_name'].iloc[tt] = os.path.join(name_raw, \
                                                                     'scramble_upto_%s.png'%\
                                                                     (image_type_names[typ]))
                    dbxpath = '/'+os.path.join(stimuli_folder,name_raw,\
                                           'scramble_upto_%s.png'%image_type_names[typ])

                # print(dbxpath)
                sys.stdout.flush()
                if dbxpath in dbxpath_dict.keys():
                    # don't have to re-create url, faster
                    url = dbxpath_dict[dbxpath]
                else:
                    url = ''
                    # print('creating dropbox url')
                    # print(dbxpath)
                    # print(dbx)
                    # url = dropbox_utils.get_shared_url(dbxpath, dbx)
                    dbxpath_dict[dbxpath] = url
                    # print(url)
                
                trial_info['dropbox_url'].iloc[tt] = url
  


proc trial 0 of 640
proc trial 100 of 640
proc trial 200 of 640
proc trial 300 of 640
proc trial 400 of 640
proc trial 500 of 640
proc trial 600 of 640


In [6]:
# figure out the sequence of which task happens when

odd_even = int(np.random.normal(0,1,1)>0)

miniblock_list = np.arange(n_mini_total)
miniblock_run = np.repeat(np.arange(n_runs), n_mini_per_run)

# miniblock tasks will always alternate, within a run
miniblock_task = np.roll(np.tile(np.arange(n_cue_levels), [int(n_mini_total/2),]), odd_even)

# flip the ordering of task within the "odd" runs
# so that every other run starts with coarse/fine miniblock
for rr in np.arange(1, n_runs, 2):
    miniblock_task[miniblock_run==rr] = np.flipud(miniblock_task[miniblock_run==rr])

In [7]:
# decide which super-category is being tested on each run of the basic task
# for the super task, this isn't relevant
miniblock_supcat = np.zeros_like(miniblock_list)
miniblock_supcat[miniblock_task==1] = -1

# want to find a way of assigning super-categories to miniblocks
# so that we don't have the same super-category twice in a run
# use brute force randomization to find good sequence
good_seq = False
max_iter = 100
ii = 0;
while (not good_seq) and (ii < max_iter):
        
    supcat_seq = np.repeat(np.arange(n_categ_use), n_mini_per_categ)
    supcat_seq = supcat_seq[np.random.permutation(len(supcat_seq))]
    miniblock_supcat[miniblock_task==0] = supcat_seq
    
    un_cats = [np.unique(miniblock_supcat[(miniblock_task==0) & (miniblock_run==rr)]) for rr in range(n_runs)]
    good_seq = np.all([len(u)==2 for u in un_cats])
    
    ii+=1

In [8]:
# now group the trials into runs based on which task is being done
            
            
# first basic-level tasks, within each superordinate   
for si, sname in enumerate(categ_use):
    
    for typ in range(n_image_types):
    
        trial_inds = np.array((trial_info['super_name']==sname) & \
                              (trial_info['cue_level']=='basic') & \
                              (trial_info['image_type_num']==typ))
    
        # name of actual basic-level category
        cue_names_actual = np.array(trial_info['basic_name'].iloc[trial_inds])
    
        # now assign the "distractor" names by shuffling real names
        distract_names = expt_utils.shuffle_nosame(cue_names_actual)
    
        trial_info['cue_name'].iloc[trial_inds] =  cue_names_actual
        trial_info['distractor_name'].iloc[trial_inds] = distract_names
    
        # randomly assign whether left or right side is the actual cue name 
        cue_on_left = np.mod(np.random.permutation(len(cue_names_actual)),2)==0
        left_name = copy.deepcopy(cue_names_actual)
        right_name = copy.deepcopy(distract_names)
        # switch half of them
        left_name[~cue_on_left] = distract_names[~cue_on_left]
        right_name[~cue_on_left] = cue_names_actual[~cue_on_left]
    
        # correct resp: 1 for left, 2 for right
        correct_resp = np.ones_like(cue_on_left).astype(int)
        correct_resp[~cue_on_left] = 2
    
        trial_info['left_name'].iloc[trial_inds] = left_name
        trial_info['right_name'].iloc[trial_inds] = right_name
    
        trial_info['correct_resp'].iloc[trial_inds] = correct_resp

    
    # now grab the trials for all image types, within this superordinate categ
    trial_inds = np.array((trial_info['super_name']==sname) & \
                              (trial_info['cue_level']=='basic'))
    
    # break all these into equal sized "mini-blocks" randomly
    # assigning to the block numbers that we already decided on previously
    miniblock_inds_use = np.repeat(miniblock_list[miniblock_supcat==si], n_trials_mini)
    run_inds_use = np.repeat(miniblock_run[miniblock_supcat==si], n_trials_mini)
    
    rand_order = np.random.permutation(len(miniblock_inds_use))
    miniblock_inds_use = miniblock_inds_use[rand_order]
    run_inds_use = run_inds_use[rand_order]

    print(miniblock_inds_use )
    trial_info['run_number'].iloc[trial_inds] = run_inds_use
    trial_info['miniblock_number_overall'].iloc[trial_inds] = miniblock_inds_use
    trial_info['miniblock_number_in_run'].iloc[trial_inds] = np.mod(miniblock_inds_use, n_mini_per_run)


[56 56 56 13 50 39 13 13 50 13 39 50 39 56 13 50 39 39 56 39 50 39 39 50
 56 56 50 13 13 56 56 39 50 13 39 13 13 50 50 56]
[31 61 31 61 61  2  2 40 61  2 40  2 31 61 31  2  2  2 40 31  2 61 31 61
 40 61 31 40 61 40 31 61 40  2 31 40 40 40 31  2]
[42 42 42 15 15 15  0  0 29 29 29 15  0 29  0 42 29  0 42 15  0 15 15 42
 29  0  0  0 15 15 42 42 42 42 15 29 29  0 29 29]
[24 24 55 45 45 55 24 55 45 24 45 34 45 55 45 45 24 34 34 24 55 55 55 34
 24 34 34 24 55 34 24 34 55 24 45 34 55 45 45 34]
[48  7 48 16 48  7 10 10 16 16 10 48 16  7 10 48  7 10  7 16 10 16 10 48
  7  7  7 16 48 16 48 48 16 16 10 48  7 10  7 10]
[37  8 21 47  8  8 37  8 47 37 37  8 21 37 21 47  8 47 37  8 37 47  8 47
 21 37 21 47 37  8 21 21 21 21 37 47 47  8 47 21]
[23 26 58 23 23 18 58 18 58 26 23 26 58 18 23 23 18 26 58 58 26 18 26 58
 18 26 23 18 26 23 58 26 18 23 58 18 26 58 23 18]
[ 5 53  5  5 32 53 63 63 63  5 63 32 53 32 63 53 32 32  5 63 32 63 53 63
  5 53 63  5 53 32 53 32 53 53  5 32 32  5  5 63]


In [9]:
# now organizing the superordinate task runs

for typ in range(n_image_types):
    
    trial_inds = np.array((trial_info['cue_level']=='super') & \
                          (trial_info['image_type_num']==typ) 
                          )
    
    # print(np.sum(trial_inds))
    
    # actual cue names (superordinate)
    cue_names_actual = np.array(trial_info['super_name'].iloc[trial_inds])
    
    # creating the list of "distractor" names that are pseudorandom
    distract_names = [np.array(categ_use)[np.array(categ_use)!=categ] for categ in categ_use]
    distract_names = [d[np.random.permutation(len(d))] for d in distract_names]
    extras = expt_utils.shuffle_nosame(np.array(categ_use))
    distract_names = [list(d)+[e] for d, e in zip(distract_names, extras)]
    distract_names = np.concatenate(distract_names, axis=0)
    
    # check they meet all the criteria
    un, counts = np.unique(distract_names, return_counts=True)
    assert(np.all(counts==counts[0]))
    assert not np.any(distract_names==cue_names_actual)
    
    trial_info['cue_name'].iloc[trial_inds] =  cue_names_actual
    trial_info['distractor_name'].iloc[trial_inds] = distract_names
    
    
    # randomly assign whether left or right side is the actual cue name 
    n_each = [np.sum(cue_names_actual==categ) for categ in categ_use]
    cue_on_left = np.concatenate([np.mod(np.random.permutation(n),2)==0 for n in n_each], \
                                 axis=0)
    left_name = copy.deepcopy(cue_names_actual)
    right_name = copy.deepcopy(distract_names)
    # switch half of them
    left_name[~cue_on_left] = distract_names[~cue_on_left]
    right_name[~cue_on_left] = cue_names_actual[~cue_on_left]
    
    # correct resp: 1 for left, 2 for right
    correct_resp = np.ones_like(cue_on_left).astype(int)
    correct_resp[~cue_on_left] = 2
    
    trial_info['left_name'].iloc[trial_inds] = left_name
    trial_info['right_name'].iloc[trial_inds] = right_name
    
    trial_info['correct_resp'].iloc[trial_inds] = correct_resp

# now grab all the trials in superordinate task, all image types
trial_inds = np.array(trial_info['cue_level']=='super')

# break all these into equal sized "mini-blocks" randomly
# assigning to the block numbers that we already decided on previously
miniblock_inds_use = np.repeat(miniblock_list[miniblock_task==1], n_trials_mini)
run_inds_use = np.repeat(miniblock_run[miniblock_task==1], n_trials_mini)


rand_order = np.random.permutation(len(miniblock_inds_use))
miniblock_inds_use = miniblock_inds_use[rand_order]
run_inds_use = run_inds_use[rand_order]

print(miniblock_inds_use)

trial_info['run_number'].iloc[trial_inds] = run_inds_use
trial_info['miniblock_number_overall'].iloc[trial_inds] = miniblock_inds_use
trial_info['miniblock_number_in_run'].iloc[trial_inds] = np.mod(miniblock_inds_use, n_mini_per_run)

[30 14 49 41 20 36 49 59 52 54 33 43 25 19 17 46 38 17 57 35 22 20  4  3
 44 59 60 35 49 49  3  4 20  1 25 17 25 28 62 62  3 14 54 12 14 12 49 20
 49 38 17 43 52  1  4 17 22 51 46 44 41 12 20 60 35 35 38 33  6 59  3  9
  9  6 11 30 35 60 28 41 52 43 57 28 11  9 51 35 22  9 59 12 54  6 14 22
 51 35  6 30 59 11 52 25  3 49 17 46 49 51 20 60 11 22  4 43 44 46  1 43
 19 33 41  4  1 22 51 49 30 11 28 60 28 19 11 19 60 52 57 25 12 33 57 57
 17 52 20 14 20 33 35  3  1 14 44  9 36  3 59 11 22 60  1 38 62 12 14 27
 27  3 54 43 28 54 44 46 30 49 38  4 52 51 62 41 19 27  9 36  9 62  6 14
 36 27 19 57 59  6 57 35 19 30 30 17 52 38  3 17 33 41  9  4  1 59 27 22
  6 27 46 20 33  3 19 11  4 62 28 44 57 44 22 36 28  6 28 36 25 38  1 30
 14 38 33 43 36 33 44 57 22 57 25 19 52 62 60 35 54 44 62 38 25 11 25 33
 46 11  1 20 19 41 43 14 30 54 38 51  9 46 41 41 51 28 27 54 62 27 62 46
 27 25 51 36  1 60 54 52 59  6 12 27 60  9 44  4 36 54 41 43  4 12 12 59
 51 43 36 46 30 17  6 12]


In [10]:
# organize the trials by mini-block number
trial_info.sort_values(by='miniblock_number_overall', inplace=True)

# now shuffle within each mini-block
for mb in np.unique(trial_info['miniblock_number_overall']):

    inds = np.where(np.array(trial_info['miniblock_number_overall']==mb))[0] 
    shuff_order = np.random.permutation(len(inds))
    trial_info.iloc[inds] = trial_info.iloc[inds[shuff_order]]
    
# assign trial numbers 
trial_info['trial_in_miniblock'] = np.tile(np.arange(n_trials_mini), [n_mini_total,])
trial_info['trial_in_run'] = np.tile(np.arange(n_trials_per_run), [n_runs,])
trial_info['trial_num_overall'] = np.arange(n_trials_total)

trial_info.set_index(np.arange(n_trials_total))


,trial_num_overall,trial_in_run,trial_in_miniblock,run_number,miniblock_number_overall,miniblock_number_in_run,image_set_num,random_order_number,categ_ind,concept_ind,...,image_type,image_name,dropbox_url,cue_level_num,cue_level,cue_name,distractor_name,left_name,right_name,correct_resp
0,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,...,pool4,raspberry_09s/scramble_upto_pool4.png,,0.0,basic,raspberry,pomegranate,pomegranate,raspberry,2.0
1,1,1,1,0.0,0.0,0.0,0.0,0.0,2.0,5.0,...,pool2,banana_05s/scramble_upto_pool2.png,,0.0,basic,banana,pear,banana,pear,1.0
2,2,2,2,0.0,0.0,0.0,0.0,0.0,2.0,1.0,...,pool2,cherry_05s/scramble_upto_pool2.png,,0.0,basic,cherry,grape,grape,cherry,2.0
3,3,3,3,0.0,0.0,0.0,0.0,0.0,2.0,4.0,...,orig,pear_01b/orig.png,,0.0,basic,pear,raspberry,raspberry,pear,2.0
4,4,4,4,0.0,0.0,0.0,0.0,0.0,2.0,2.0,...,pool2,raspberry_05s/scramble_upto_pool2.png,,0.0,basic,raspberry,apple,apple,raspberry,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,635,35,5,15.0,63.0,3.0,0.0,0.0,7.0,2.0,...,orig,airplane_01b/orig.png,,0.0,basic,airplane,motorcycle,airplane,motorcycle,1.0
636,636,36,6,15.0,63.0,3.0,0.0,0.0,7.0,3.0,...,pool4,truck_09n/scramble_upto_pool4.png,,0.0,basic,truck,motorcycle,motorcycle,truck,2.0
637,637,37,7,15.0,63.0,3.0,0.0,0.0,7.0,1.0,...,pool1,train_03s/scramble_upto_pool1.png,,0.0,basic,train,airplane,airplane,train,2.0
638,638,38,8,15.0,63.0,3.0,0.0,0.0,7.0,7.0,...,pool4,canoe_09s/scramble_upto_pool4.png,,0.0,basic,canoe,airplane,canoe,airplane,1.0


In [13]:
check_trial_info(trial_info, concepts_use, categ_use)

In [23]:
ti = trial_info

In [16]:
trial_info[np.mod(trial_info['trial_in_run'],10)==0][0:32]

,trial_num_overall,trial_in_run,trial_in_miniblock,run_number,miniblock_number_overall,miniblock_number_in_run,image_set_num,random_order_number,categ_ind,concept_ind,...,image_type,image_name,dropbox_url,cue_level_num,cue_level,cue_name,distractor_name,left_name,right_name,correct_resp
188,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,...,pool4,raspberry_09s/scramble_upto_pool4.png,,0.0,basic,raspberry,pomegranate,pomegranate,raspberry,2.0
107,10,10,0,0.0,1.0,1.0,0.0,0.0,3.0,6.0,...,pool2,milkshake_06s/scramble_upto_pool2.png,,1.0,super,dessert,vehicle,vehicle,dessert,2.0
92,20,20,0,0.0,2.0,2.0,0.0,0.0,1.0,3.0,...,pool1,onion_03n/scramble_upto_pool1.png,,0.0,basic,onion,cabbage,cabbage,onion,2.0
413,30,30,0,0.0,3.0,3.0,0.0,0.0,0.0,6.0,...,orig,moth_02n/orig.png,,1.0,super,insect,dessert,dessert,insect,2.0
423,40,0,0,1.0,4.0,0.0,0.0,0.0,0.0,6.0,...,pool1,moth_04n/scramble_upto_pool1.png,,1.0,super,insect,fruit,fruit,insect,2.0
578,50,10,0,1.0,5.0,1.0,0.0,0.0,7.0,0.0,...,pool3,ship_07s/scramble_upto_pool3.png,,0.0,basic,ship,train,train,ship,2.0
137,60,20,0,1.0,6.0,2.0,0.0,0.0,5.0,6.0,...,pool3,clarinet_08s/scramble_upto_pool3.png,,1.0,super,musical_instrument,tool,tool,musical_instrument,2.0
322,70,30,0,1.0,7.0,3.0,0.0,0.0,4.0,0.0,...,pool1,pencil_03s/scramble_upto_pool1.png,,0.0,basic,pencil,hammer,pencil,hammer,1.0
414,80,0,0,2.0,8.0,0.0,0.0,0.0,5.0,1.0,...,orig,guitar_01b/orig.png,,0.0,basic,guitar,clarinet,clarinet,guitar,2.0
603,90,10,0,2.0,9.0,1.0,0.0,0.0,4.0,5.0,...,pool3,shovel_08n/scramble_upto_pool3.png,,1.0,super,tool,insect,tool,insect,1.0


In [12]:

def check_trial_info(ti, concepts_use, categ_use):

    # double checking all the trial attributes
    
    
    # check that we assigned the "correct response" correctly
    # 1 is left, 2 is right
    inds_check = np.array(ti['correct_resp']==1)
    assert(np.all(ti['cue_name'][inds_check]==ti['left_name'][inds_check]))
    inds_check = np.array(ti['correct_resp']==2)
    assert(np.all(ti['cue_name'][inds_check]==ti['right_name'][inds_check]))

    # cue and distractor name always different
    assert(not np.any(ti['cue_name']==ti['distractor_name']))

    # check that basic and super task are assigned correctly
    inds_check = np.array(ti['cue_level']=='basic')
    assert(np.all(np.isin(np.array(ti['cue_name'].iloc[inds_check]), concepts_use)))
    assert(np.all(np.isin(np.array(ti['distractor_name'].iloc[inds_check]), concepts_use)))

    # for each basic block checking that the cue and distractor names are all part of a single superord categ
    for mb in np.unique(ti['miniblock_number_overall'].iloc[inds_check]):
        inds_check_mb = inds_check & (np.array(ti['miniblock_number_overall']==mb))
        sname = np.array(ti['super_name'].iloc[inds_check_mb])
        assert(np.all(sname==sname[0]))
        sind = np.where(np.array(categ_use)==sname[0])[0][0]
        assert(np.all(np.isin(np.array(ti['distractor_name'].iloc[inds_check_mb]), concepts_use[sind])))
        assert(np.all(np.isin(np.array(ti['cue_name'].iloc[inds_check_mb]), concepts_use[sind])))

    # check superordinate names 
    inds_check = np.array(ti['cue_level']=='super')
    assert(np.all(np.isin(np.array(ti['cue_name'].iloc[inds_check]), categ_use)))
    assert(np.all(np.isin(np.array(ti['distractor_name'].iloc[inds_check]), categ_use)))

    # check to make sure individual attributes are distributed evenly across trials 
    n_trials_total = ti.shape[0]

    attr_check_even = ['super_name', 'basic_name', 'ex_num', 'image_type_num', \
                       'correct_resp', 'cue_level', 'run_number', \
                       'miniblock_number_overall', 'miniblock_number_in_run']
    for attr in attr_check_even:

        # should be an equal number of each thing 
        un, counts = np.unique(ti[attr], return_counts=True)
        # print(un, counts)
        assert(np.all(counts==n_trials_total/len(un)))


    # check the counterbalancing over multiple attributes

    # there should be an equal number of trials in each of the combinations of these 
    # different attribute "levels". for example each combination of category/image type. 

    attr_balanced = [ti['categ_ind'], ti['concept_ind'], ti['image_type_num'], ti['cue_level']]
    attr_balanced_inds = np.array([np.unique(attr, return_inverse=True)[1] for attr in attr_balanced]).T

    n_levels_each = [len(np.unique(attr)) for attr in attr_balanced]
    n_combs_expected = np.prod(n_levels_each)
    n_repeats_expected = n_trials_total/n_combs_expected

    un_rows, counts = np.unique(attr_balanced_inds,axis=0, return_counts=True)

    assert(un_rows.shape[0]==n_combs_expected)
    assert(np.all(counts==n_repeats_expected))

    
    
    # check evenness of stuff for super task only

    inds_check = np.array(ti['cue_level']=='super')
    n_trials_total = np.sum(inds_check)

    attr_check_even = ['super_name', 'cue_name', 'image_type', 'distractor_name', 'correct_resp']

    for attr in attr_check_even:

        # should be an equal number of each thing 
        un, counts = np.unique(ti[attr].iloc[inds_check], return_counts=True)
        # print(un, counts)
        assert(np.all(counts==n_trials_total/len(un)))



    # check evenness of stuff for basic task only

    inds_check = np.array(ti['cue_level']=='basic')
    n_trials_total = np.sum(inds_check)

    attr_check_even = ['basic_name', 'cue_name', 'image_type', 'distractor_name', 'correct_resp']

    for attr in attr_check_even:

        # should be an equal number of each thing 
        un, counts = np.unique(ti[attr].iloc[inds_check], return_counts=True)
        # print(un, counts)
        assert(np.all(counts==n_trials_total/len(un)))

    # making sure that for the different superordinate tasks, 
    # the miniblocks all fall on different runs
    for si, sname in enumerate(categ_use):
        
        inds = (ti['cue_level']=='basic') & (ti['super_name']==sname)
        assert(len(np.unique(ti['run_number'][inds])) == n_mini_per_categ)

In [14]:
trial_info

,trial_num_overall,trial_in_run,trial_in_miniblock,run_number,miniblock_number_overall,miniblock_number_in_run,image_set_num,random_order_number,categ_ind,concept_ind,...,image_type,image_name,dropbox_url,cue_level_num,cue_level,cue_name,distractor_name,left_name,right_name,correct_resp
214,0,0,0,0.0,0.0,0.0,0.0,0.0,2.0,5.0,...,pool1,banana_03s/scramble_upto_pool1.png,,0.0,basic,banana,apple,apple,banana,2.0
212,1,1,1,0.0,0.0,0.0,0.0,0.0,2.0,4.0,...,orig,pear_01b/orig.png,,0.0,basic,pear,raspberry,raspberry,pear,2.0
172,2,2,2,0.0,0.0,0.0,0.0,0.0,2.0,1.0,...,pool2,cherry_05s/scramble_upto_pool2.png,,0.0,basic,cherry,grape,grape,cherry,2.0
234,3,3,3,0.0,0.0,0.0,0.0,0.0,2.0,1.0,...,pool1,cherry_03s/scramble_upto_pool1.png,,0.0,basic,cherry,pear,cherry,pear,1.0
184,4,4,4,0.0,0.0,0.0,0.0,0.0,2.0,3.0,...,pool2,apple_05s/scramble_upto_pool2.png,,0.0,basic,apple,raspberry,apple,raspberry,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
129,635,35,5,15.0,63.0,3.0,0.0,0.0,4.0,1.0,...,pool1,knife_04s/scramble_upto_pool1.png,,1.0,super,tool,dessert,tool,dessert,1.0
117,636,36,6,15.0,63.0,3.0,0.0,0.0,0.0,4.0,...,pool3,caterpillar_08s/scramble_upto_pool3.png,,1.0,super,insect,vehicle,insect,vehicle,1.0
271,637,37,7,15.0,63.0,3.0,0.0,0.0,1.0,3.0,...,pool3,onion_08s/scramble_upto_pool3.png,,1.0,super,vegetable,vehicle,vehicle,vegetable,2.0
41,638,38,8,15.0,63.0,3.0,0.0,0.0,3.0,3.0,...,orig,cupcake_02s/orig.png,,1.0,super,dessert,vegetable,dessert,vegetable,1.0


In [114]:
# break all these into equal sized "mini-blocks" randomly
# assigning to the block numbers that we already decided on previously

miniblock_inds_use = np.repeat(miniblock_list[miniblock_supcat==si], n_trials_mini)
run_inds_use = np.repeat(miniblock_run[miniblock_supcat==si], n_trials_mini)

rand_order = np.random.permutation(len(miniblock_inds_use))
miniblock_inds_use = miniblock_inds_use[rand_order]
run_inds_use = run_inds_use[rand_order]

trial_info['run_number'].iloc[trial_inds] = run_inds_use
trial_info['miniblock_number_overall'].iloc[trial_inds] = miniblock_inds_use
trial_info['miniblock_number_in_run'].iloc[trial_inds] = np.mod(miniblock_inds_use, n_mini_per_run)


In [113]:
trial_info

,trial_num_overall,run_number,miniblock_number_overall,miniblock_number_in_run,trial_in_run,image_set_num,random_order_number,categ_ind,concept_ind,super_name,...,image_type,image_name,dropbox_url,cue_level_num,cue_level,cue_name,distractor_name,left_name,right_name,correct_resp
0,0.0,8.0,32.0,0.0,0.0,0.0,0.0,0.0,0.0,insect,...,orig,beetle_01b/orig.png,,0.0,basic,beetle,caterpillar,beetle,caterpillar,1.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,insect,...,orig,beetle_02n/orig.png,,1.0,super,0.0,0.0,0.0,0.0,0.0
2,2.0,3.0,14.0,2.0,0.0,0.0,0.0,0.0,0.0,insect,...,pool1,beetle_03s/scramble_upto_pool1.png,,0.0,basic,beetle,mosquito,mosquito,beetle,2.0
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,insect,...,pool1,beetle_04s/scramble_upto_pool1.png,,1.0,super,0.0,0.0,0.0,0.0,0.0
4,4.0,2.0,10.0,2.0,0.0,0.0,0.0,0.0,0.0,insect,...,pool2,beetle_05n/scramble_upto_pool2.png,,0.0,basic,beetle,grasshopper,grasshopper,beetle,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,635.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,vehicle,...,pool2,canoe_06s/scramble_upto_pool2.png,,1.0,super,0.0,0.0,0.0,0.0,0.0
636,636.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,vehicle,...,pool3,canoe_07s/scramble_upto_pool3.png,,0.0,basic,0.0,0.0,0.0,0.0,0.0
637,637.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,vehicle,...,pool3,canoe_08s/scramble_upto_pool3.png,,1.0,super,0.0,0.0,0.0,0.0,0.0
638,638.0,0.0,0.0,0.0,0.0,0.0,0.0,7.0,7.0,vehicle,...,pool4,canoe_09s/scramble_upto_pool4.png,,0.0,basic,0.0,0.0,0.0,0.0,0.0


In [ ]:
# break all these into equal sized "mini-blocks" randomly
basic_mini_seq = np.repeat(super_run_order, int(n_trials_per_run/n_image_types))
super_run_seq = super_run_seq[np.random.permutation(len(super_run_seq))]
trial_info['run_number'].iloc[trial_inds] = super_run_seq


trial_info['run_number'].iloc[trial_inds] = run_seq_rand


In [90]:
np.sum(trial_inds)

8

In [ ]:

# now group the trials into runs based on which task is being done

# first basic-level tasks, within each superordinate   
for si, sname in enumerate(categ_use):

    for typ in range(n_image_types):

        trial_inds = np.array((trial_info['super_name']==sname) & \
                              (trial_info['cue_level']=='basic') & \
                              (trial_info['image_type_num']==typ))

        # name of actual basic-level category
        cue_names_actual = np.array(trial_info['basic_name'].iloc[trial_inds])

        # now assign the "distractor" names by shuffling real names
        distract_names = expt_utils.shuffle_nosame(cue_names_actual)

        trial_info['cue_name'].iloc[trial_inds] =  cue_names_actual
        trial_info['distractor_name'].iloc[trial_inds] = distract_names

        # randomly assign whether left or right side is the actual cue name 
        cue_on_left = np.mod(np.random.permutation(len(cue_names_actual)),2)==0
        left_name = copy.deepcopy(cue_names_actual)
        right_name = copy.deepcopy(distract_names)
        # switch half of them
        left_name[~cue_on_left] = distract_names[~cue_on_left]
        right_name[~cue_on_left] = cue_names_actual[~cue_on_left]

        # correct resp: 1 for left, 2 for right
        correct_resp = np.ones_like(cue_on_left).astype(int)
        correct_resp[~cue_on_left] = 2

        trial_info['left_name'].iloc[trial_inds] = left_name
        trial_info['right_name'].iloc[trial_inds] = right_name

        trial_info['correct_resp'].iloc[trial_inds] = correct_resp

        # break all these into equal sized "mini-blocks" randomly
        basic_mini_seq = np.repeat(super_run_order, int(n_trials_per_run/n_image_types))
        super_run_seq = super_run_seq[np.random.permutation(len(super_run_seq))]
        trial_info['run_number'].iloc[trial_inds] = super_run_seq


        trial_info['run_number'].iloc[trial_inds] = run_seq_rand


In [35]:
basic_miniblock_order = np.arange(n_categ_use * n_mini_per_categ) * 2 + odd_even

In [33]:
basic_miniblock_order

array([51, 35, 39, 41, 57, 17, 59, 55, 15,  3, 19, 43, 27, 23,  5,  9, 13,
       53, 11, 61, 31, 29,  1, 63, 49, 37, 25, 47, 21,  7, 33, 45])

In [28]:
n_trials_per_concept

10

In [27]:
n_categ_use

8

In [26]:
basic_miniblock_order

array([ 0,  8, 12,  2,  6,  4, 10, 14])

In [24]:
super_run_order

array([ 1,  3,  5,  7,  9, 11, 13, 15])

In [23]:
basic_run_order

array([14, 12,  4,  8,  6,  0,  2, 10])

In [ ]:
# randomly deciding whether basic or super happens on odd/even runs
odd_even = int(np.random.normal(0,1,1)>0)
basic_run_order = np.random.permutation(len(categ_use))*2 + odd_even
super_run_order = np.arange(n_runs)
super_run_order = super_run_order[~np.isin(super_run_order, basic_run_order)]



In [14]:
trial_info

,trial_num_overall,run_number,trial_in_run,image_set_num,random_order_number,categ_ind,concept_ind,super_name,basic_name,ex_num,...,image_type,image_name,dropbox_url,cue_level_num,cue_level,cue_name,distractor_name,left_name,right_name,correct_resp
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,insect,beetle,0.0,...,orig,beetle_01b/orig.png,,0.0,basic,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,insect,beetle,1.0,...,orig,beetle_02n/orig.png,,1.0,super,0.0,0.0,0.0,0.0,0.0
2,2.0,0.0,0.0,0.0,0.0,0.0,0.0,insect,beetle,2.0,...,pool1,beetle_03s/scramble_upto_pool1.png,,0.0,basic,0.0,0.0,0.0,0.0,0.0
3,3.0,0.0,0.0,0.0,0.0,0.0,0.0,insect,beetle,3.0,...,pool1,beetle_04s/scramble_upto_pool1.png,,1.0,super,0.0,0.0,0.0,0.0,0.0
4,4.0,0.0,0.0,0.0,0.0,0.0,0.0,insect,beetle,4.0,...,pool2,beetle_05n/scramble_upto_pool2.png,,0.0,basic,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
635,635.0,0.0,0.0,0.0,0.0,7.0,7.0,vehicle,canoe,5.0,...,pool2,canoe_06s/scramble_upto_pool2.png,,1.0,super,0.0,0.0,0.0,0.0,0.0
636,636.0,0.0,0.0,0.0,0.0,7.0,7.0,vehicle,canoe,6.0,...,pool3,canoe_07s/scramble_upto_pool3.png,,0.0,basic,0.0,0.0,0.0,0.0,0.0
637,637.0,0.0,0.0,0.0,0.0,7.0,7.0,vehicle,canoe,7.0,...,pool3,canoe_08s/scramble_upto_pool3.png,,1.0,super,0.0,0.0,0.0,0.0,0.0
638,638.0,0.0,0.0,0.0,0.0,7.0,7.0,vehicle,canoe,8.0,...,pool4,canoe_09s/scramble_upto_pool4.png,,0.0,basic,0.0,0.0,0.0,0.0,0.0
